In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import scipy.io.wavfile
(rate,snd)=scipy.io.wavfile.read("268074__skowm001__3-tone-chime.wav")
plt.plot(snd)


plot the frequency domain graph of wav
<br>
fs_rate, signal = wavfile.read("268074__skowm001__3-tone-chime.wav")<br>
print ("Frequency sampling", fs_rate)<br>
l_audio = len(signal.shape)<br>
print ("Channels", l_audio)<br>
if l_audio == 2:<br>
    signal = signal.sum(axis=1) / 2<br>
N = signal.shape[0]<br>
print ("Complete Samplings N", N)<br>
secs = N / float(fs_rate)<br>
print ("secs", secs)<br>
Ts = 1.0/fs_rate # sampling interval in time<br>
print ("Timestep between samples Ts", Ts)<br>
t = scipy.arange(0, secs, Ts) # time vector as scipy arange field / numpy.ndarray<br>
FFT = abs(scipy.fft(signal))<br>
FFT_side = FFT[range(N//2)] # one side FFT range<br>
freqs = scipy.fftpack.fftfreq(signal.size, t[1]-t[0])<br>
fft_freqs = np.array(freqs)<br>
freqs_side = freqs[range(N//2)] # one side frequency range<br>
fft_freqs_side = np.array(freqs_side)<br>
plt.subplot(311)<br>
p1 = plt.plot(t, signal, "g") # plotting the signal<br>
plt.xlabel('Time')<br>
plt.ylabel('Amplitude')<br>
plt.subplot(312)<br>
p2 = plt.plot(freqs, FFT, "r") # plotting the complete fft spectrum<br>
plt.xlabel('Frequency (Hz)')<br>
plt.ylabel('Count dbl-sided')<br>
plt.subplot(313)<br>
p3 = plt.plot(freqs_side, abs(FFT_side), "b") # plotting the positive fft spectrum<br>
plt.xlabel('Frequency (Hz)')<br>
plt.ylabel('Count single-sided')<br>
plt.show()<br>
ssize=2**16<br>
fwin=np.hamming(ssize)<br>
ff=np.fft.rfft(snd[:ssize])<br>
def ffunc(ix,cutoff):<br>
    if ix < cutoff:<br>
        return 1.0<br>
    return 0.0<br>
filt=[ffunc(ix,len(ff)/20) for ix in range(len(ff))]<br>
plt.plot(filt)<br>
plt.show()<br>
ff=[ff[ix]*filt[ix] for ix in range(len(ff))]<br>
fsamp=np.fft.irfft(ff)<br>
plt.plot(fsamp[:ssize])<br>
plt.show()<br>
filt_res=np.array([])<br>
def fft_slice(startloc,fracbw):<br>
    global filt_res<br>
    ff=np.fft.rfft(snd[startloc:ssize+startloc])<br>
    filt=[ffunc(ix,len(ff)/fracbw) for ix in range(len(ff))]<br>
    ff=[ff[ix]*filt[ix] for ix in range(len(ff))]<br>
    fsamp=np.fft.irfft(ff)<br>
    filt_res=np.append(filt_res,fsamp)<br>
filt_res=np.array([])<br>
for startit in range(0,len(snd),ssize):<br>
    fft_slice(startit,50)<br>
plt.plot(filt_res)<br>
plt.show()<br>
sf=max(snd)<br>
filt_scale=[filt_res[ix]/sf for ix in range(len(filt_res))]<br>
filt_scale=np.array(filt_scale)<br>
scipy.io.wavfile.write("changed_268074__skowm001__3-tone-chime.wav",rate,filt_scale)<br>
filt_scale[3]<br>
lot the changed wav


In [ ]:
fs_rate, signal = wavfile.read("changed_268074__skowm001__3-tone-chime.wav")
print ("Frequency sampling", fs_rate)
l_audio = len(signal.shape)
print ("Channels", l_audio)
if l_audio == 2:
    signal = signal.sum(axis=1) / 2
N = signal.shape[0]
print ("Complete Samplings N", N)
secs = N / float(fs_rate)
print ("secs", secs)
Ts = 1.0/fs_rate # sampling interval in time
print ("Timestep between samples Ts", Ts)
t = scipy.arange(0, secs, Ts) # time vector as scipy arange field / numpy.ndarray
FFT = abs(scipy.fft(signal))
FFT_side = FFT[range(N//2)] # one side FFT range
freqs = scipy.fftpack.fftfreq(signal.size, t[1]-t[0])
fft_freqs = np.array(freqs)
freqs_side = freqs[range(N//2)] # one side frequency range
fft_freqs_side = np.array(freqs_side)
plt.subplot(311)
p1 = plt.plot(t, signal, "g") # plotting the signal
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.subplot(312)
p2 = plt.plot(freqs, FFT, "r") # plotting the complete fft spectrum
plt.xlabel('Frequency (Hz)')
plt.ylabel('Count dbl-sided')
plt.subplot(313)
p3 = plt.plot(freqs_side, abs(FFT_side), "b") # plotting the positive fft spectrum
plt.xlabel('Frequency (Hz)')
plt.ylabel('Count single-sided')
plt.show()


convert changed wav to csv
<br>
input_filename = "changed_268074__skowm001__3-tone-chime.wav"<br>
if input_filename[-3:] != 'wav':<br>
    print('WARNING!! Input File format should be *.wav')<br>
    sys.exit()<br>
samrate, data = wavfile.read(str(input_filename))<br>
print('Load is Done! \n')<br>
wavData = pd.DataFrame(data)<br>
if len(wavData.columns) == 2:<br>
    print('Stereo .wav file\n')<br>
    wavData.columns = ['R', 'L']<br>
    stereo_R = pd.DataFrame(wavData['R'])<br>
    stereo_L = pd.DataFrame(wavData['L'])<br>
    print('Saving...\n')<br>
    stereo_R.to_csv(str(input_filename[:-4] + "_Output_stereo_R.csv"), mode='w')<br>
    stereo_L.to_csv(str(input_filename[:-4] + "_Output_stereo_L.csv"), mode='w')<br>
    # wavData.to_csv("Output_stereo_RL.csv", mode='w')<br>
    print('Save is done ' + str(input_filename[:-4]) + '_Output_stereo_R.csv , '<br>
                          + str(input_filename[:-4]) + '_Output_stereo_L.csv')<br>
elif len(wavData.columns) == 1:<br>
    print('Mono .wav file\n')<br>
    wavData.columns = ['M']<br>
    wavData.to_csv(str(input_filename[:-4] + "_Output_mono.csv"), mode='w')<br>
    print('Save is done ' + str(input_filename[:-4]) + '_Output_mono.csv')<br>
else:<br>
    print('Multi channel .wav file\n')<br>
    print('number of channel : ' + len(wavData.columns) + '\n')<br>
    wavData.to_csv(str(input_filename[:-4] + "Output_multi_channel.csv"), mode='w')<br>
    print('Save is done ' + str(input_filename[:-4]) + 'Output_multi_channel.csv')<br>
plt.show()